# Negation Correction

In [ ]:
!pip install spacy
!pip install scispacy
!pip install negspacy

!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_ner_bc5cdr_md-0.5.3.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_core_sci_sm-0.5.3.tar.gz

In [2]:
## Imports
import spacy
import scispacy
from spacy import displacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from negspacy.negation import Negex

In [3]:
nlp0 = spacy.load("en_core_sci_sm")
nlp1 = spacy.load("en_ner_bc5cdr_md")

/usr/local/lib/python3.10/dist-packages/spacy/language.py:2141: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [4]:
#lemmatizing the notes to capture all forms of negation(e.g., deny: denies, denying)
def lemmatize(nt):
    doc = nlp0(nt)
    lemNote = [wd.lemma_ for wd in doc]
    return " ".join(lemNote)

In [16]:
from spacy import displacy

# clinical_note = "Admit note (0030-0700) 02:30 Pt admit via ED from OSH, where presented from work with low sats. Had 10 days of SOB unresolved with mucinex and albuterol. PMH of ETOH, smoking, PVD with claudication, asthma (takes Albuterol regularly) CT done at OSH, found to have large saddle PE, started on Heparin at 1400 units/hr with PTT now in therapeutic range. Tx to SICU for closer VS monitoring. Currently: Neuro. A+Ox3, no defecits noted, no complaints of pain. Resp. On 2l nc with SpO2 95-98%, LS clear to all fields, quiet to UL's. CVS. HR 80's NSR, no ectopy, am labs wnl. BP 120/80. NS at 125cc/hr. GI/GU. pt able to take regular diet, +BS, [**Last Name (un) **] soft, non distended. pt voiding to urinal. Skin. all areas intact. Plan. Echo. EKG. Sinus rhythm. Non-diagnostic repolarization abnormalities. No previous tracing available for comparison. PATIENT/TEST INFORMATION: Indication: Pulmonary embolus. Right ventricular function. Height: (in) 75 Weight (lb): 178 BSA (m2): 2.09 m2 BP (mm Hg): 117/69 HR (bpm): 95 Status: Inpatient Date/Time: [**2158-2-3**] at 12:24 Test: Portable TTE (Complete) Doppler: Full Doppler and color Doppler Contrast: None Technical Quality: Adequate INTERPRETATION: Findings: LEFT ATRIUM: Mild LA enlargement. RIGHT ATRIUM/INTERATRIAL SEPTUM: Normal RA size. LEFT VENTRICLE: Normal LV wall thickness. Normal LV cavity size. Depressed LVEF."
clinical_note = "pulmonary emboli No contraindications for IV contrast"
lem_clinical_note= lemmatize(clinical_note)
print(lem_clinical_note)

#creating a doc object using BC5CDR model
# doc = nlp1(lem_clinical_note)

# #function to modify options for displacy NER visualization
# def get_entity_options():
#     entities = ["DISEASE", "NEG_ENTITY", "CHEMICAL"]
#     colors = {'DISEASE': 'linear-gradient(180deg, #66ffcc, #abf763)', 'CHEMICAL': 'linear-gradient(90deg, #aa9cfc, #fc9ce7)', "NEG_ENTITY":'linear-gradient(90deg, #ffff66, #ff6600)'}
#     options = {"ents": entities, "colors": colors}
#     return options

# options = get_entity_options()
# #visualizing identified Named Entities in clinical input text
# displacy.render(doc, style='ent', jupyter=True, options=options)

pulmonary embolus no contraindication for iv contrast


In [ ]:
from negspacy.termsets import termset

#adding a new pipeline component to identify negation
nlp = spacy.load("en_ner_bc5cdr_md", disable = ['parser'])
nlp.add_pipe('sentencizer')
# negex = Negex(nlp)
# nlp.add_pipe(negex)

ts = termset("en_clinical")

# nlp.add_pipe("negex", config={"ent_types":["DISEASE"]})
nlp.add_pipe(
  "negex",
  config={
      "neg_termset":ts.get_patterns()
  })


In [17]:
import re


"""
Negspacy sets a new attribute e._.negex to True if a negative concept is encountered
"""
def contains_negation(input_note):
    refined_note = re.sub(r"[!@#$%^&*;:/<>?\|`\-=\_+]", ".", input_note)

    note= lemmatize(refined_note)

    results = []
    note = note.split(".") #sentence tokenizing based on delimeter
    note = [n.strip() for n in note] #removing extra spaces at the begining and end of sentence
    has_neg = False
    for t in note:
      doc = nlp(t)
      for e in doc.ents:
        rs = str(e._.negex)
        print("text: ", e.text)
        print("negex: ", e._.negex)
        if 'pulmonary embol' in e.text.lower() or e.text.lower() == 'pe':
          has_neg = (rs == "True")
    return has_neg

contains_negation(clinical_note)

# #list of negative concepts from clinical note identified by negspacy
# results0 = negation_handling(clinical_note)

text:  pulmonary embolus
negex:  False


False

In [ ]:
from spacy import displacy

clinical_note = "Admit note (0030-0700) 02:30 Pt admit via ED from OSH, where presented from work with low sats. Had 10 days of SOB unresolved with mucinex and albuterol. PMH of ETOH, smoking, PVD with claudication, asthma (takes Albuterol regularly) CT done at OSH, found to have large saddle PE, started on Heparin at 1400 units/hr with PTT now in therapeutic range. Tx to SICU for closer VS monitoring. Currently: Neuro. A+Ox3, no defecits noted, no complaints of pain. Resp. On 2l nc with SpO2 95-98%, LS clear to all fields, quiet to UL's. CVS. HR 80's NSR, no ectopy, am labs wnl. BP 120/80. NS at 125cc/hr. GI/GU. pt able to take regular diet, +BS, [**Last Name (un) **] soft, non distended. pt voiding to urinal. Skin. all areas intact. Plan. Echo. EKG. Sinus rhythm. Non-diagnostic repolarization abnormalities. No previous tracing available for comparison. PATIENT/TEST INFORMATION: Indication: Pulmonary embolus. Right ventricular function. Height: (in) 75 Weight (lb): 178 BSA (m2): 2.09 m2 BP (mm Hg): 117/69 HR (bpm): 95 Status: Inpatient Date/Time: [**2158-2-3**] at 12:24 Test: Portable TTE (Complete) Doppler: Full Doppler and color Doppler Contrast: None Technical Quality: Adequate INTERPRETATION: Findings: LEFT ATRIUM: Mild LA enlargement. RIGHT ATRIUM/INTERATRIAL SEPTUM: Normal RA size. LEFT VENTRICLE: Normal LV wall thickness. Normal LV cavity size. Depressed LVEF. LV dysnchrony is present. RIGHT VENTRICLE: Normal RV wall thickness. Markedly dilated RV cavity. Severe global RV free wall hypokinesis. Abnormal septal motion/position. AORTA: Normal aortic root diameter. Normal ascending aorta diameter. AORTIC VALVE: Normal aortic valve leaflets (3). No AS. MITRAL VALVE: Mildly thickened mitral valve leaflets. TRICUSPID VALVE: Moderate to severe [3+] TR. Moderate PA systolic hypertension. PERICARDIUM: No pericardial effusion. Conclusions: 1. The left atrium is mildly dilated. 2. Left ventricular wall thicknesses are normal. The left ventricular cavity size is normal. LV systolic function appears depressed, at least moderately and possibly severe. The views are limited to make an accurate assessment. Left ventricular dysnchrony is present. 3.The right ventricular cavity is markedly dilated. There is severe global right ventricular free wall hypokinesis. There is abnormal septal motion/position. 4.The aortic valve leaflets (3) appear structurally normal with good leaflet excursion. There is no aortic valve insufficiency seen. 5.The mitral valve leaflets are mildly thickened. No mitral regurgitation seen. 6. Moderate to severe [3+] tricuspid regurgitation is seen. 7.There is moderate pulmonary artery systolic hypertension. 8.There is no pericardial effusion."
lem_clinical_note= lemmatize(clinical_note)

#creating a doc object using BC5CDR model
doc = nlp1(lem_clinical_note)

#function to modify options for displacy NER visualization
def get_entity_options():
    entities = ["DISEASE", "NEG_ENTITY", "CHEMICAL"]
    colors = {'DISEASE': 'linear-gradient(180deg, #66ffcc, #abf763)', 'NEG_ENTITY': 'linear-gradient(90deg, #aa9cfc, #fc9ce7)', "CHEMICAL":'linear-gradient(90deg, #ffff66, #ff6600)'}
    options = {"ents": entities, "colors": colors}
    return options

options = get_entity_options()
#visualizing identified Named Entities in clinical input text
displacy.render(doc, style='ent', jupyter=True, options=options)

In [37]:
def compare_dates(row, given_date):
    return 1 if row['CHARTDATE'] >= given_date else 0

def check_PE(input_text):
  if ("pulmonary embol" in input_text.lower()) or (" pe " in input_text.lower()):
    return not contains_negation(input_text)
  return False

In [34]:
#@title notes

# clinical_note = "Admit note (0030-0700) 02:30 Pt admit via ED from OSH, where presented from work with low sats. Had 10 days of SOB unresolved with mucinex and albuterol. PMH of ETOH, smoking, PVD with claudication, asthma (takes Albuterol regularly) CT done at OSH, found to have large saddle PE, started on Heparin at 1400 units/hr with PTT now in therapeutic range. Tx to SICU for closer VS monitoring. Currently: Neuro. A+Ox3, no defecits noted, no complaints of pain. Resp. On 2l nc with SpO2 95-98%, LS clear to all fields, quiet to UL's. CVS. HR 80's NSR, no ectopy, am labs wnl. BP 120/80. NS at 125cc/hr. GI/GU. pt able to take regular diet, +BS, [**Last Name (un) **] soft, non distended. pt voiding to urinal. Skin. all areas intact. Plan. Echo. EKG. Sinus rhythm. Non-diagnostic repolarization abnormalities. No previous tracing available for comparison. PATIENT/TEST INFORMATION: Indication: Pulmonary embolus. Right ventricular function. Height: (in) 75 Weight (lb): 178 BSA (m2): 2.09 m2 BP (mm Hg): 117/69 HR (bpm): 95 Status: Inpatient Date/Time: [**2158-2-3**] at 12:24 Test: Portable TTE (Complete) Doppler: Full Doppler and color Doppler Contrast: None Technical Quality: Adequate INTERPRETATION: Findings: LEFT ATRIUM: Mild LA enlargement. RIGHT ATRIUM/INTERATRIAL SEPTUM: Normal RA size. LEFT VENTRICLE: Normal LV wall thickness. Normal LV cavity size. Depressed LVEF. LV dysnchrony is present. RIGHT VENTRICLE: Normal RV wall thickness. Markedly dilated RV cavity. Severe global RV free wall hypokinesis. Abnormal septal motion/position. AORTA: Normal aortic root diameter. Normal ascending aorta diameter. AORTIC VALVE: Normal aortic valve leaflets (3). No AS. MITRAL VALVE: Mildly thickened mitral valve leaflets. TRICUSPID VALVE: Moderate to severe [3+] TR. Moderate PA systolic hypertension. PERICARDIUM: No pericardial effusion. Conclusions: 1. The left atrium is mildly dilated. 2. Left ventricular wall thicknesses are normal. The left ventricular cavity size is normal. LV systolic function appears depressed, at least moderately and possibly severe. The views are limited to make an accurate assessment. Left ventricular dysnchrony is present. 3.The right ventricular cavity is markedly dilated. There is severe global right ventricular free wall hypokinesis. There is abnormal septal motion/position. 4.The aortic valve leaflets (3) appear structurally normal with good leaflet excursion. There is no aortic valve insufficiency seen. 5.The mitral valve leaflets are mildly thickened. No mitral regurgitation seen. 6. Moderate to severe [3+] tricuspid regurgitation is seen. 7.There is moderate pulmonary artery systolic hypertension. 8.There is no pericardial effusion."

clinical_note_2639_157999 = "[**2113-10-1**] 9:59 AM CHEST (PORTABLE AP) Clip # [**Clip Number (Radiology) 77129**] Reason: please eval for pna, masses. The patient is febrile with a c Admitting Diagnosis: CHEST PAIN ______________________________________________________________________________ [**Hospital 2**] MEDICAL CONDITION: 64 year old woman with COPD, CAD, depression, CHF now with low grade fever and chronic cough. REASON FOR THIS EXAMINATION: please eval for pna, masses. The patient is febrile with a cough (dx of COPD) ______________________________________________________________________________ FINAL REPORT CLINICAL HISTORY: COPD, CHF, now with low-grade fevers. CHEST AP: The heart shows some left ventricular prominence and calcification of the aorta. The diaphragms are somewhat flattened, consistent with known COPD. No acute lung process is identified. No evidence of failure or pneumonia is seen. There has been little change since the prior chest x-ray of [**2112-10-7**]. IMPRESSION: No failure, no evidence of pneumonia. [**2113-10-1**] 3:35 PM CHEST (PORTABLE AP); -76 BY SAME PHYSICIAN [**Name Initial (PRE) 5**] # [**Clip Number (Radiology) 77161**] Reason: ? ETT placement Admitting Diagnosis: CHEST PAIN ______________________________________________________________________________ [**Hospital 2**] MEDICAL CONDITION: 64 year old woman with respiratory distress new intubation REASON FOR THIS EXAMINATION: ? ETT placement ______________________________________________________________________________ FINAL REPORT CLINICAL HISTORY: Respiratory distress, newly intubated, check position. The tip of the endotracheal tube lies at the level of the clavicles approximately 9 cm from the carinal angle and should therefore be advanced. The tip of the nasogastric tube lies opposite to the eighth vertebral body within the esophagus. New opacities are present at both bases and there is suggestion of Kerley B lines, which could indicate failure rather than pneumonia as the cause. IMPRESSION: Endotracheal tube and nasogastric tube should be advanced. Evidence of acute onset of failure. [**2113-10-1**] 6:26 PM CHEST PORT. LINE PLACEMENT; -77 BY DIFFERENT PHYSICIAN [**Name Initial (PRE) 5**] # [**Clip Number (Radiology) 77052**] Reason: ? ETT tube and OG tube placement, new right femoral line Admitting Diagnosis: CHEST PAIN ______________________________________________________________________________ [**Hospital 2**] MEDICAL CONDITION: 64 year old woman with respiratory distress REASON FOR THIS EXAMINATION: ? ETT tube and OG tube placement, new right femoral line ______________________________________________________________________________ WET READ: KKgc SUN [**2113-10-1**] 8:51 PM ET tube has been advanced, now ends approximately 6 cm above carina. NG tube in the stomach. Bibasal opacities with [**Last Name (un) 1392**] lines, suggestive of edema, unchanged. Small left pleural effusion. ______________________________________________________________________________ FINAL REPORT HISTORY: Tube placement. FINDINGS: In comparison with the earlier study of this date, there has been placement of a nasogastric tube that is coiled within the upper stomach. Endotracheal tube tip has been advanced and now lies approximately 6 cm above the carina. Little change in the appearance of the heart and lungs. [**2113-10-1**] 8:33 PM CTA CHEST W&W/O C&RECONS, NON-CORONARY Clip # [**Clip Number (Radiology) 77053**] Reason: ? pulmonary embolist Admitting Diagnosis: CHEST PAIN Field of view: 36 Contrast: OPTIRAY Amt: 100 ______________________________________________________________________________ [**Hospital 2**] MEDICAL CONDITION: 64 year old woman with shock, hypoxemia and tachycardia REASON FOR THIS EXAMINATION: ? pulmonary embolist No contraindications for IV contrast ______________________________________________________________________________ WET READ: KKgc SUN [**2113-10-1**] 9:38 PM 1. No evidence of pulmonary embolism or acute aortic pathology. Moderate coronary artery calcification. 2. Multifocal heterogeneously enhancing consolidations in both lower lobes, minimally in the lingula and right middle lobe, likely represent multifocal pneumonia. Prominent septal lines suggest mild pulmonary edema. Centrilobular emphysema. Small prevascular lymphnodes. 3. ETT ends 5 cm above carina. NGT coils in the stomach and courses out of view. Kkaliann. ______________________________________________________________________________ FINAL REPORT HISTORY: 64-year-old with shock, hypoxemia and tachycardia, therefore, question pulmonary embolus. TECHNIQUE: Multidetector CT imaging was performed from the thoracic inlet through to the hemidiaphragms. This was initially performed prior to intravenous contrast, followed by contrast infusion with bolus timing to maximize pulmonary arterial enhancement. CT OF THE CHEST WITH AND WITHOUT CONTRAST, FINDINGS: There is no evidence of a pulmonary embolus. There is prominence of the pulmonary arteries, raising the question of pulmonary hypertension, however. Additionally, there is enlargement of the right ventricle as well as dilatation of the IVC, all indicative of an element of right heart failure. Lung parenchyma demonstrates diffuse bilateral patchy infiltrates including almost complete collapse of the lower lobes. There are small free layering bilateral pleural effusions. There is increase in interseptal markings, suggesting an element of cardiac failure. There are scattered areas of parenchymal loss also, indicative of emphysema. Endotracheal tube shows its tip in the mid trachea, NG tube is seen coiled within the stomach. Markedly limited evaluation of the liver and spleen, no definite gross abnormalities. There is atherosclerotic disease involving the descending aorta. There are some prominent lymph nodes within the mediastinum but not meeting size criteria for pathologic enlargement. Bone windows demonstrate no suspicious areas. IMPRESSION: 1. No evidence of pulmonary embolus. (Over) [**2113-10-1**] 8:33 PM CTA CHEST W&W/O C&RECONS, NON-CORONARY Clip # [**Clip Number (Radiology) 77053**] Reason: ? pulmonary embolist Admitting Diagnosis: CHEST PAIN Field of view: 36 Contrast: OPTIRAY Amt: 100 ______________________________________________________________________________ FINAL REPORT (Cont) 2. Diffuse patchy bilateral infiltrates as well as extensive volume loss of the lower lobes bilaterally. Only small bilateral pleural effusions. 2. Prominence of the pulmonary arteries, suggesting element of pulmonary hypertension. 3. Right ventricular enlargement and right IVC enlargement, raising the question of right heart failure. This would be in [**Location (un) 11438**] with the increase in interstitial markings suggesting an element of failure. Sinus tachycardia with a single aberrantly conducted beat. Biatrial abnormality. Compared to the previous tracing the sinus rate is faster. The other findings are similar. TRACING #2 Sinus tachycardia. Biatrial abnormality. Compared to the previous tracing the findings are similar. TRACING #3 Sinus rhythm. Slightly leftward axis. Non-specific ST-T wave abnormalities. Compared to the previous tracing the rhythm is more clearly sinus today. TRACING #1 PATIENT/TEST INFORMATION: Indication: Congestive heart failure. Left ventricular function. Height: (in) 67 Weight (lb): 112 BSA (m2): 1.58 m2 BP (mm Hg): 97/62 HR (bpm): 96 Status: Inpatient Date/Time: [**2113-10-1**] at 20:13 Test: TTE (Focused views) Doppler: Limited Doppler and color Doppler Contrast: None Technical Quality: Adequate INTERPRETATION: Findings: LEFT ATRIUM: Elongated LA. RIGHT ATRIUM/INTERATRIAL SEPTUM: Mildly dilated RA. LEFT VENTRICLE: Moderate-severe regional left ventricular systolic dysfunction. Severely depressed LVEF. [Intrinsic LV systolic function likely depressed given the severity of valvular regurgitation.] LV WALL MOTION: Regional LV wall motion abnormalities include: mid anterior - hypo; mid anteroseptal - hypo; basal inferoseptal - hypo; mid inferoseptal - hypo; basal inferior - hypo; mid inferior - hypo; anterior apex - hypo; septal apex - hypo; inferior apex - hypo; apex - hypo; RIGHT VENTRICLE: Normal RV chamber size and free wall motion. AORTA: Normal descending aorta diameter. AORTIC VALVE: Normal aortic valve leaflets (?#). No AR. MITRAL VALVE: Mildly thickened mitral valve leaflets. Moderate (2+) MR. TRICUSPID VALVE: Normal tricuspid valve leaflets. Mild to moderate [[**1-8**]+] TR. PERICARDIUM: No pericardial effusion. GENERAL COMMENTS: Emergency study performed by the cardiology fellow on call. Conclusions: The left atrium is elongated. There is moderate to severe regional left ventricular systolic dysfunction with severe hypokinesis of the inferior, septal, anterior and apical walls. Overall left ventricular systolic function is severely depressed (LVEF= 25-30 %). [Intrinsic left ventricular systolic function is likely more depressed given the severity of valvular regurgitation.] Right ventricular chamber size and free wall motion are normal. The aortic valve leaflets (?#) appear structurally normal with good leaflet excursion. No aortic regurgitation is seen. The mitral valve leaflets are mildly thickened. Moderate (2+) mitral regurgitation is seen. There is no pericardial effusion. IMPRESSION: Suboptimal image quality. Extensive regional left ventricular systolic dysfunction c/w multivessel CAD or other diffuse process. Moderate mitral regurgitation. No pericardial effusion. Compared with the prior study (images reviewed) of [**2113-9-30**], the inferior wall is now hypokinetic with increased mitral regurgitation c/w interim ischemia.	"

clinical_note_18718_162712 = "Admit note (0030-0700) 02:30 Pt admit via ED from OSH, where presented from work with low sats. Had 10 days of SOB unresolved with mucinex and albuterol. PMH of ETOH, smoking, PVD with claudication, asthma (takes Albuterol regularly) CT done at OSH, found to have large saddle PE, started on Heparin at 1400 units/hr with PTT now in therapeutic range. Tx to SICU for closer VS monitoring. Currently: Neuro. A+Ox3, no defecits noted, no complaints of pain. Resp. On 2l nc with SpO2 95-98%, LS clear to all fields, quiet to UL's. CVS. HR 80's NSR, no ectopy, am labs wnl. BP 120/80. NS at 125cc/hr. GI/GU. pt able to take regular diet, +BS, [**Last Name (un) **] soft, non distended. pt voiding to urinal. Skin. all areas intact. Plan. Echo. EKG. Sinus rhythm. Non-diagnostic repolarization abnormalities. No previous tracing available for comparison. PATIENT/TEST INFORMATION: Indication: Pulmonary embolus. Right ventricular function. Height: (in) 75 Weight (lb): 178 BSA (m2): 2.09 m2 BP (mm Hg): 117/69 HR (bpm): 95 Status: Inpatient Date/Time: [**2158-2-3**] at 12:24 Test: Portable TTE (Complete) Doppler: Full Doppler and color Doppler Contrast: None Technical Quality: Adequate INTERPRETATION: Findings: LEFT ATRIUM: Mild LA enlargement. RIGHT ATRIUM/INTERATRIAL SEPTUM: Normal RA size. LEFT VENTRICLE: Normal LV wall thickness. Normal LV cavity size. Depressed LVEF. LV dysnchrony is present. RIGHT VENTRICLE: Normal RV wall thickness. Markedly dilated RV cavity. Severe global RV free wall hypokinesis. Abnormal septal motion/position. AORTA: Normal aortic root diameter. Normal ascending aorta diameter. AORTIC VALVE: Normal aortic valve leaflets (3). No AS. MITRAL VALVE: Mildly thickened mitral valve leaflets. TRICUSPID VALVE: Moderate to severe [3+] TR. Moderate PA systolic hypertension. PERICARDIUM: No pericardial effusion. Conclusions: 1. The left atrium is mildly dilated. 2. Left ventricular wall thicknesses are normal. The left ventricular cavity size is normal. LV systolic function appears depressed, at least moderately and possibly severe. The views are limited to make an accurate assessment. Left ventricular dysnchrony is present. 3.The right ventricular cavity is markedly dilated. There is severe global right ventricular free wall hypokinesis. There is abnormal septal motion/position. 4.The aortic valve leaflets (3) appear structurally normal with good leaflet excursion. There is no aortic valve insufficiency seen. 5.The mitral valve leaflets are mildly thickened. No mitral regurgitation seen. 6. Moderate to severe [3+] tricuspid regurgitation is seen. 7.There is moderate pulmonary artery systolic hypertension. 8.There is no pericardial effusion."

clinical_note_17977_198062 = "[**2159-10-27**] 9:09 AM CHEST (PORTABLE AP) Clip # [**Clip Number (Radiology) 18264**] Reason: ?PNA, Pneumothorax Admitting Diagnosis: LEFT HUMERUS FRACTURE/SDA ______________________________________________________________________________ [**Hospital 2**] MEDICAL CONDITION: 69 year old woman with new SOB (Patient is in hemodilaysis on [**Hospital Ward Name 246**] 7) REASON FOR THIS EXAMINATION: ?PNA, Pneumothorax ______________________________________________________________________________ FINAL REPORT CHEST ONE VIEW PORTABLE INDICATION: 69-year-old woman with shortness of breath. COMMENTS: Portable semi-erect AP radiograph of the chest is reviewed, and compared with the previous study dated [**2159-10-25**]. The lung volume is small. Patchy atelectasis is seen at the lung bases. The lungs are clear otherwise. The heart is normal in size. There is continued tortuosity of the thoracic aorta. The right jugular IV catheter remains in place. No pneumothorax is identified. [**2159-10-27**] 10:18 AM CHEST (PORTABLE AP); -76 BY SAME PHYSICIAN [**Name Initial (PRE) 42**] # [**Clip Number (Radiology) 18265**] Reason: s/p intubation; please eval for interval change Admitting Diagnosis: LEFT HUMERUS FRACTURE/SDA ______________________________________________________________________________ [**Hospital 2**] MEDICAL CONDITION: 69 year old woman with new SOB (Patient is in hemodilaysis on [**Hospital Ward Name 246**] 7); now with PEA arrest REASON FOR THIS EXAMINATION: s/p intubation; please eval for interval change ______________________________________________________________________________ FINAL REPORT CHEST ONE VIEW PORTABLE. INDICATION: 69-year-old woman with new shortness of breath. COMMENTS: Portable supine AP radiograph of the chest is reviewed, and compared with the previous study of 9:42 a.m. The tip of the endotracheal tube is identified just above the carina. The subsequent chest radiograph demonstrated adjustment of the location of the endotracheal tube. The right jugular IV catheter terminates in the distal SVC. No pneumothorax is seen. The lungs are clear. The heart is normal in size. Again, note is made of marked tortuosity of the thoracic aorta. Patient has prior left shoulder prosthesis. Note is made of dilatation of the stomach. [**2159-10-27**] 1:15 PM CHEST PORT. LINE PLACEMENT; -76 BY SAME PHYSICIAN [**Name Initial (PRE) 42**] # [**Clip Number (Radiology) 18266**] Reason: s/p L sub clav unsuccessful attempt, now with R femoral A li Admitting Diagnosis: LEFT HUMERUS FRACTURE/SDA ______________________________________________________________________________ [**Hospital 2**] MEDICAL CONDITION: 69 year old woman with new SOB (Patient is in hemodilaysis on [**Hospital Ward Name 246**] 7); now with PEA arrest, s/p L subclavian attempt. REASON FOR THIS EXAMINATION: s/p L sub clav unsuccessful attempt, now with R femoral A line and L femoral venous line ______________________________________________________________________________ FINAL REPORT INDICATION: 69-year-old woman with shortness of breath. COMMENTS: Portable supine AP radiograph of the chest is reviewed, and compared with the previous study at 10:37 a.m. The tip of the endotracheal tube is identified 2 cm above the carina. A nasogastric tube terminates in the gastric fundus. The previously identified dilated stomach has been improved. The right jugular IV catheter remains in place. The lung volume is small. The lungs are clear. The heart is normal in size. There is continued tortuosity of the thoracic aorta. The patient has prior left shoulder prosthesis. No pneumothorax is identified. resp care pt received from dialysis s/p PEA arrest. ett taped at 23 lip, pulled back to 21 per cxray. initially requiring high ve, multiple adjustments made d/t air trapping,dysynchrony,high plateaus, presently stable on present settings of ac 500x14/40/5. c/w full support. Nsg Adm Note Mrs.[**Known lastname 4613**] is a 69yo woman emergently adm to MICU s/p Resp/cardiac arrest in HD. PMH:DM, ESRD:on HD, Diabetic Neuropathy,Left Femur fx:bed bound, Hyponatremia,Hypercholesterolemia,Cataracts,back pain, HTN, Anemia Meds:hydromorphone, Insulin,folic acid,nephrocaps, Glipizide All: ASA,losartan.lisinopril, HCTZ Pt was initially adm to floor on [**10-24**] for left humoral FX repair which she had on [**10-25**] tol well and was getting ready for trandfer back home. She went for her HD this am where she was half way into treatment when she c/o Acute SOB, she was given some O2 but upon attempt to get ABG she became unrsp and code called. She recieved EPi, atropine, IVF intubated and transfered here. MICU Adm course: CV:pt was emergently adm to MCIU B, her Hr was in the 140's and her BP was in the 60's,She was given lg amts of IVF via pressure (total of 5L given) and started on NEO. She was titrated up quickly but BP slow to rsp so VASOPRESSIN was addded. An Arterial line was attempted but proved difficult to get, eventually a femoral A-line was placed. Her BP has improved through the day and by 6pm, pt has been weaned off [**Last Name (un) **] drugs. Her BP is still maintaing a MAP of 60 88-94/50's. Her HR has dropped nicely off the pressors and in less pain(hydromorphone was given). Pt did require the total of 5L of IVF that she was given and she is now having all fluid minimized. House staff still ? MI as her EKG shows some ischemia ?rate/stress related, CPK and troponin pnd. Stat ECHO was unequivocal Resp:pt was intubated during the code,initially we were unable to obtain any O2 sats or ABG. Her vent was maximized per Resp therapy and her eventual ABG was much improved. She has min secretions. The house staff feels she may have had a PE so she will eventually go for a CTA. We have been able to wean her Fio2 down to 40% but cont on A/C 500x14 5 PEEP.Her CXR shows vol overload now. Still unable to get O2 sats. Heme:pt's hct has been stable so far. She was started on Heparin drip at 4pm for ?PE vs ACS. She did have OB- stool and but OB trace + ngt asp. No other obvious bleeding. PTT needed at 10pm GU:difficult to pass foley due to prev surgical/cultural intervention, but no urine was obtained. Bladder scan also showed no urine there, therefore n foley. Per family she makes infreq urine. GI: OGT placed, sm amt of asp OB+ noted, hypoactive bowel sounds, lg amt of loose stool, rectal bag applied. Family reports 2-4 months of poor appetite overall, only apple juice!! Neuro:pt was unrsp upon adm. She began to open eyes to noxiuos stimulus but it wasn't until her family talked to her (3pm) that she rsp to simple commands-she only speaks Somali. She was aprop prior to arrest. She was medicated for pain of shoulder at 4pm which she rsp well to 1mg of hydromorphome ID: pt has been afebrile, cx with blood,sputum and stool unable tot get urine. Vanco level pnd from prev dose prior to surgery. Her HD catheter was just placed in [**Month (only) 212**]. She had a new fem Aline and Quad cath placed here. Her WBC ar Nsg Adm Note (Continued) e up to 35 with 16 bands ENDO:pt was placed on a Insulin drip for aggressive control, she is currently on 1u/hr, last BS 143. Skin:Pt with withered rt toes and poor arterial perfusion. She does have dopplerable pulse but DP are not as strong as the PT are. She moves her feet when touched. A/P:Will keep MAP >60 and Support as needed, avoid furhter fluid Wean vent as able to get ABG to pt's norm, pulm toilet as needed Await cx results, follow fever curve Watch for bleeding, PTT at 10pm Medicate for pain as much as needed Note stool amt to be eval for HD daily [**2159-10-27**] 9:42 PM CHEST (PORTABLE AP); -77 BY DIFFERENT PHYSICIAN [**Name Initial (PRE) 1**] # [**Clip Number (Radiology) 98348**] Reason: r/o PTX Admitting Diagnosis: LEFT HUMERUS FRACTURE/SDA ______________________________________________________________________________ [**Hospital 3**] MEDICAL CONDITION: 69 year old woman with new SOB (Patient is in hemodilaysis on [**Hospital Ward Name **] 7); now with PEA arrest, s/p L subclavian attempt. REASON FOR THIS EXAMINATION: r/o PTX ______________________________________________________________________________ FINAL REPORT HISTORY: Shortness of breath. Single portable chest radiograph demonstrates an endotracheal tube with the tip at the level of the clavicular heads. A feeding tube is present with its tip in the stomach. Right subclavian central venous catheter is present with its tip at the atrial-caval junction. Lordotic positioning limits assessment slightly. Ill-defined opacity overlying the right mid lung may represent a structure outside the patient. Costophrenic angles are sharp. The lungs are otherwise clear. Lung volumes are low. Trachea is midline. The patient is status post left shoulder arthroplasty. No significant change when compared to chest radiograph obtained earlier the same day. [**2159-10-27**] 11:24 PM CTA CHEST W&W/O C &RECONS; CT 100CC NON IONIC CONTRAST Clip # [**Clip Number (Radiology) 18268**] Reason: TACHY.HTN.ASYSTOLIC ARREST.R/O PE Admitting Diagnosis: LEFT HUMERUS FRACTURE/SDA Field of view: 36 Contrast: VISAPAQUE Amt: 100 ______________________________________________________________________________ [**Hospital 2**] MEDICAL CONDITION: 69 year old woman with asystolic arrest, afterwards tachycardic and hypotensive REASON FOR THIS EXAMINATION: evaluate for PE No contraindications for IV contrast ______________________________________________________________________________ FINAL REPORT INDICATION: Asystolic arrest with tachycardia and hypotension, evaluate for pulmonary embolus. COMPARISON: [**2159-7-26**]. TECHNIQUE: Axial MDCT images were obtained through the chest prior to and following the administration of 100 cc of intravenous Visipaque in the pulmonary arterial phase. Coronal and sagittal reformations are provided. CONTRAST: Intravenous nonionic contrast was administered due to the rapid rate of bolus injection required for this examination and due to elevated patient creatinine, Visipaque was administered. CTA OF THE CHEST WITHOUT AND WITH INTRAVENOUS CONTRAST: There is a filling defect within the right lower lobe pulmonary artery consistent with pulmonary embolus. No other definite filling defects are identified within the pulmonary arteries. There is no pathologic appearing mediastinal, hilar, or axillary lymphadenopathy. The central airways appear patent. The ascending aorta is ectatic. There are aortic mural calcifications consistent with atheromatous disease. The heart and pericardium appear unremarkable. There is a small left pleural effusion and left lower lobe atelectasis. Minimal dependent change within the right upper and right lower lobes. Ill- defined 2 mm nodule within the left base (series 3 image 12). No pneumothorax. Limited images of the upper abdomen demonstrate reflux of contrast into the hepatic veins. There are punctate calcifications within the spleen suggestive of prior granulomatous disease. Note is made of thickening of the left pectoralis muscle and stranding along the pectoralis fascia on the left. There is a suggestion of soft tissue stranding about the left shoulder. Evaluation is limited by the presence of streak artifact arising from the left humeral prosthesis. BONE WINDOWS: Bone windows demonstrate no evidence of suspicious lytic or sclerotic osseous lesions. Questionable fracture of the right third rib laterally and of the left fourth and sixth ribs anteriorly. MULTIPLANAR REFORMATS: Coronal and sagittal reformations demonstrate a (Over) [**2159-10-27**] 11:24 PM CTA CHEST W&W/O C &RECONS; CT 100CC NON IONIC CONTRAST Clip # [**Clip Number (Radiology) 18268**] Reason: TACHY.HTN.ASYSTOLIC ARREST.R/O PE Admitting Diagnosis: LEFT HUMERUS FRACTURE/SDA Field of view: 36 Contrast: VISAPAQUE Amt: 100 ______________________________________________________________________________ FINAL REPORT (Cont) filling defect in the right lower lobe pulmonary artery consistent with a pulmonary embolus. There is a compression deformity of a lower thoracic vertebra. IMPRESSION: 1. Right lower lobe pulmonary embolus. This finding was called to Dr. [**Last Name (STitle) 7680**] at the time of interpretation. 2. Small left pleural effusion. 3. Probable nondisplaced bilateral rib fractures. 4. 2 mm nodule at the left lung base. This is probably unchanged from [**7-26**], [**2159**]. If there is no history of malignancy, one-year CT followup may be obtained to determine stability. PATIENT/TEST INFORMATION: Indication: Left ventricular function. Right ventricular function. S/p PEA arrest Height: (in) 65 Weight (lb): 180 BSA (m2): 1.89 m2 BP (mm Hg): 86/48 HR (bpm): 150 Status: Inpatient Date/Time: [**2159-10-27**] at 12:34 Test: Portable TTE (Focused views) Doppler: Limited Doppler and no color Doppler Contrast: None Technical Quality: Adequate INTERPRETATION: Findings: LEFT VENTRICLE: Normal LV wall thickness, cavity size, and systolic function (LVEF>55%). Suboptimal technical quality, a focal LV wall motion abnormality cannot be fully excluded. RIGHT VENTRICLE: RV not well seen. AORTIC VALVE: Mildly thickened aortic valve leaflets. MITRAL VALVE: Mildly thickened mitral valve leaflets. TRICUSPID VALVE: Normal tricuspid valve leaflets. Normal PA systolic pressure. PULMONIC VALVE/PULMONARY ARTERY: Pulmonic valve not well seen. PERICARDIUM: No pericardial effusion. GENERAL COMMENTS: Suboptimal image quality - poor echo windows. Emergency study performed by the cardiology fellow on call. Conclusions: Left ventricular wall thickness, cavity size, and systolic function are normal (LVEF>55%). Due to suboptimal technical quality, a focal wall motion abnormality cannot be fully excluded. The aortic valve leaflets are mildly thickened. The mitral valve leaflets are mildly thickened. The estimated pulmonary artery systolic pressure is normal. There is no pericardial effusion. Sinus rhythm Generalized low QRS voltages Since previous tracing, rate slower"

clinical_note_1066_154406 = "[**2197-12-25**] 1:59 PM CHEST (PA & LAT) Clip # [**Clip Number (Radiology) 96612**] Reason: r/o infectious vs pe ______________________________________________________________________________ [**Hospital 2**] MEDICAL CONDITION: 65 year old woman with sob and continues low sat on ozs/p pe diagnosis 1 week pta REASON FOR THIS EXAMINATION: r/o infectious vs pe ______________________________________________________________________________ FINAL REPORT PA AND LATERAL CHEST, [**2197-12-25**] AT 14:00 HOURS HISTORY: Shortness of breath and hypoxia with known PE. COMPARISON: No prior studies are available at this Institution. FINDINGS: Lung volumes are markedly diminished with resultant bronchovascular crowding in the lung bases. There is subtle increased opacity involving primarily the left lung base with multiple areas of linear subsegmental atelectasis. The remaining lungs are clear. The mediastinum is unremarkable. The cardiac silhouette is within normal limits accounting for patient and technical factors. No pleural effusion or pneumothorax is evident. The osseous structures are unremarkable. IMPRESSION: Left lower lobe opacity likely due to atelectasis, although an early developing pneumonia or possibly a pulmonary infarction in light of given history cannot be entirely excluded. [**2197-12-25**] 3:27 PM CTA CHEST W&W/O C&RECONS, NON-CORONARY Clip # [**Clip Number (Radiology) 96613**] Reason: eval: clot burden Field of view: 40 ______________________________________________________________________________ [**Hospital 2**] MEDICAL CONDITION: 65 year old woman with known PE, worsening hypoxia REASON FOR THIS EXAMINATION: eval: clot burden No contraindications for IV contrast ______________________________________________________________________________ WET READ: KMcd MON [**2197-12-25**] 4:55 PM 1. Multiple partially occlusive second order branch PEs in both lower lobes and right upper lobe. 2. Small left pleural effusion. 3. Bilateral dependent atelectasis, left greater than right. WET READ VERSION #1 KMcd MON [**2197-12-25**] 4:10 PM ______________________________________________________________________________ FINAL REPORT INDICATION: 65-year-old woman with known PE, worsening hypoxia. Evaluate for clot burden. COMPARISONS: No prior CTs are available for comparison purposes on PACS. TECHNIQUE: CTA of the chest without and with contrast. Multiplanar reformations were performed. FINDINGS: There are several incomplete central filling defects in segmental (4th-order) pulmonary arterial branches of both right upper and right lower lobe (3:22, 3:24 and 3:46). An incomplete, central filling defect is also seen in a segmental branch of left lower lobe pulmonary artery branch (3:51). No main pulmonary artery or saddle embolus is seen. Several lymph nodes are seen in the lower paratracheal and prevascular stations, not meeting CT size criteria for pathologic enlargement. There is no axillary lymphadenopathy. There is a small left-sided pleural effusion. There is moderate dependent atelectasis on the left and mild on the right. No consolidations are seen to suggest pneumonia. The heart is normal in size. There is no pericardial effusion. Atherosclerotic calcifications are seen in the aortic arch, origin of great vessels and the LAD. Images of the upper abdomen do not reveal acute pathological findings. The adrenal glands appear unremarkable. BONE WINDOWS: No suspicious lytic or blastic lesions are seen. IMPRESSION: 1. Partially occlusive PEs in segmental (4th-order) pulmonary artery branches in the right upper and both lower lobes, as described. Comparison with a prior outside hospital CTA, if available, would be necessary to assess for interval evolution. 2. Small left pleural effusion. 3. Dependent atelectatic changes, moderate on the left and mild on the right. (Over) [**2197-12-25**] 3:27 PM CTA CHEST W&W/O C&RECONS, NON-CORONARY Clip # [**Clip Number (Radiology) 96613**] Reason: eval: clot burden Field of view: 40 FINAL REPORT (Cont) Sinus rhythm Anterior T wave changes are nonspecific No previous tracing available for comparison"

clinical_note = "Admission Date: [**2112-10-3**] Discharge Date: [**2112-10-11**] Date of Birth: [**2044-3-4**] Sex: F Service: MEDICINE Allergies: Patient recorded as having No Known Allergies to Drugs Attending:[**First Name3 (LF) 3913**] Chief Complaint: DOE x 6 weeks Major Surgical or Invasive Procedure: none History of Present Illness: 68 year-old F with multiple myeloma, on Thalidomide and Decadron who presents with progressive DOE x 6 weeks. She used to be able to walk up one flight of stairs without SOB and today she was unable to walk to her bathroom. On the morning of admission, she also reported left-sided chest pain that radiated to her back. She had accompanying dizziness and nausea/vomiting. No pleuritic or positional components to the pain. She denies SOB at rest, orthopnea, or PND. No change in her chronic LE edema. Of note she was taken off her diuretics (for mild renal failure) 6 weeks ago, but had to restart them 3 weeks ago due to worsened LE edema. She also recently traveled in a car to VA for the Labor Day weekend. . In the ED she was hypoxic to the low 90's on 100% NRB. Her lactate was elevated at 3.4, SBP 95, and out of concern for sepsis, she received 1.2 L of IVF. Once her BNP returned at 4948, her IVF was stopped and she put out 1200cc to 80 IV lasix. She also received a dose of levoflox and was placed on BIPAP. ABG on 60% FiO2: 7.42/47/72. Her EKG was unchanged, cardiac enzymes were negative. Past Medical History: Multiple Myeloma - Dx in [**2108**] with IgA Kappa monoclonal protein, followed without treatment. In [**2112-3-30**], she was noted to have an IgA level of 2300, with increased fatigue. Bone Marrow Bx revealed > 50% plasma cells qualifying her for the Vaccine Study, for which she is enrolled. She received 3 cycles of velcade. She has been receiving Zometa and started on Thalidomide and Decadron ([**2112-5-30**]). 2. Hypertension. 3. Hypercholesterolemia. 4. Hyperglycemia on Decadron 5. Restrictive lung dz: by PFTs in [**4-4**] 6. Chronic back pain (s/p MVA) 7. Duodenal ulcer 11 years ago. 8. History of Lichen simplex chronicus. 9. Right knee pain: s/p cortisone injections Social History: She is married and lives with her husband. She has three children. She works as a high school librarian. Denies tobacco use; drinks EtOH socially. Family History: One daughter has sarcoidosis. No FH of MI or CVA Physical Exam: Vitals: T: 96.7 P: 92 BP: 107/30 RR: 21 SaO2: 96% on BiPAP: 12/5/0.60 General: Awake, alert, NAD on BiPAP mask. HEENT: PERRL, EOMI, sclera anicteric. Neck: thick, JVD to clavicle. no carotid bruits appreciated, 2+ carotid pulses. Pulm: Lungs clear bilaterally, occ crackles to b/t bases Cardiac: RRR, nl S1/S2, no M/R/G appreciated Abdomen: soft, NT/ND, + BS, no hepatomegaly noted. Rectal (in ED): guaiac negative brown stool Ext: 1+ edema b/t, 2+ radial, DP pulses b/l. Neurologic: Alert & Oriented x 3. Able to relate history without difficulty. Pertinent Results: [**2112-10-3**] 01:45PM PT-12.7 PTT-24.8 INR(PT)-1.1 [**2112-10-3**] 01:45PM PLT COUNT-265 [**2112-10-3**] 01:45PM ANISOCYT-1+ [**2112-10-3**] 01:45PM NEUTS-88.3* LYMPHS-8.4* MONOS-2.9 EOS-0.3 BASOS-0.1 [**2112-10-3**] 01:45PM WBC-14.8*# RBC-3.51* HGB-10.8* HCT-31.6* MCV-90 MCH-30.9 MCHC-34.4 RDW-16.9* [**2112-10-3**] 01:45PM CRP-167.3* [**2112-10-3**] 01:45PM CORTISOL-50.7* [**2112-10-3**] 01:45PM CALCIUM-9.0 PHOSPHATE-4.4# MAGNESIUM-1.9 [**2112-10-3**] 01:45PM CK-MB-NotDone proBNP-4948* [**2112-10-3**] 01:45PM cTropnT-0.07* [**2112-10-3**] 01:45PM LIPASE-53 [**2112-10-3**] 01:45PM ALT(SGPT)-16 AST(SGOT)-16 CK(CPK)-89 ALK PHOS-57 AMYLASE-152* TOT BILI-0.3 [**2112-10-3**] 01:45PM GLUCOSE-227* UREA N-42* CREAT-3.4*# SODIUM-137 POTASSIUM-4.0 CHLORIDE-93* TOTAL CO2-28 ANION GAP-20 [**2112-10-3**] 01:55PM LACTATE-3.8* K+-4.1 [**2112-10-3**] 04:10PM URINE BLOOD-NEG NITRITE-NEG PROTEIN-NEG GLUCOSE-NEG KETONE-NEG BILIRUBIN-NEG UROBILNGN-NEG PH-5.0 LEUK-NEG [**2112-10-3**] 04:10PM URINE COLOR-Yellow APPEAR-Clear SP [**Last Name (un) 155**]-1.014 [**2112-10-3**] 05:38PM PO2-72* PCO2-47* PH-7.42 TOTAL CO2-32* BASE XS-4 INTUBATED-NOT INTUBA [**2112-10-3**] 10:41PM %HbA1c-9.2* [Hgb]-DONE [A1c]-DONE [**2112-10-3**] 10:41PM ALBUMIN-3.3* [**2112-10-3**] 10:41PM CK-MB-4 cTropnT-0.06* [**2112-10-3**] 10:41PM CK(CPK)-74 [**2112-10-3**] 11:04PM LACTATE-1.1 [**2112-10-3**] 11:04PM TYPE-ART PO2-56* PCO2-48* PH-7.44 TOTAL CO2-34* BASE XS-6 INTUBATED-NOT INTUBA [**2112-10-3**] 11:29PM TYPE-ART PO2-95 PCO2-44 PH-7.45 TOTAL CO2-32* BASE XS-5 EKG: NSR @ 85, nl axis/intervals, no ST or T wave changes. . <b>Radiologic Data: CXR [**10-3**]: Mild cardiomegaly, low lung volumes, but no evidence for pneumonia. . PFTs [**4-/2112**]: Reduced FVC (72%) and Dsb (63%) with low-normal TLC suggests a restrictive process. . Echo [**3-/2112**]: 1. The left atrium is mildly dilated. 2. Left ventricular wall thicknesses are normal. The LV cavity size is normal. Regional LV wall motion is normal. Overall left ventricular systolic function is normal (LVEF> 55%). Tissue velocity imaging demonstrates an E/e' <8 suggesting a normal LV filling pressure. 3. Right ventricular chamber size is normal. RV systolic function is normal. 4.The aortic valve leaflets are mildly thickened. There is no valvular aortic stenosis. The increased transaortic gradient is likely related to high cardiac output. No aortic regurgitation is seen. 5.The mitral valve leaflets are mildly thickened. No mitral regurgitation is seen. 6.The estimated pulmonary artery systolic pressure could not be estimated because of the lack of a tricusped regurgitation jet. 7.There is no pericardial effusion. Brief Hospital Course: 68 yo F with multiple myeloma on Thalidomide and Decadron who presents with progressive DOE x 6 weeks, hypoxia, hypotension, and ARF. The following issues were investigated during her hospitalization: 1) Resp Distress: On presentation to the CCU, the patient still was still hypoxic to 80s on room air, requiring 60% face mask. Her physical exam was unremarkable for CHF - she had no peripheral edema, no JVD and her lungs were clear bilaterally. Given her hypoxia, elevated A-a gradient (297), history of multiple myeloma and her treatment with Thalidomide and Decadron (known to be thrombogenic and pt. was not on outpatient anticoagulation), there was significant concern for a PE. Her Thalidomide was discontinued and she was started on a Heparin drip. Her elevated creatinine did not allow for a CTA so a VQ scan was performed and was highly suspcious for a PE with multiple perfusion deficits. Additionally, an echo showed depressed RV function and evidence of RV hypertrophy and LE dopplers showed a DVT in her right popliteal artery. The evidence of RV hypertrophy and multiple VQ mismatches suggested an acute and chronic thromboembolic disease. The Heparin drip was continued and she was started on Coumadin. As she rapidly improved she was not thought to be a candidate for thrombolytics. Her aPTT goal was 60-80 while she was transitioned to Coumadin. Upon arrival to the BMT service, her hypoxia continued to improve. By time of discharge, she maintained normal oxygen saturation on room air. As the patient had transient risk factors for DVT/PE (i.e. recent long car ride and thalidomide), will anticipate ~6 months of anticoagulation with INR [**3-4**]. . 2.) CARDIAC: * Ischemia: On admission to the ED, the patient had CE drawn to r/o an MI. CK = 89 and Trop = 0.07. There is no reported history of CAD and no EKG changes, but the patient has hyperlipidemia, HTN, and hyperglycemia. She was continued on ASA 81 mg. Metoprolol 12.5 was added once her BP could tolerate it. A statin should perhaps be considered given an LDL in [**March 2112**] of 131; this can be revisited as an outpatient. . * Pump: Pt's pro-BNP on admission was 4948. She also has a history of chronic LE edema for which she takes Lasix and Triamterine as an outpatient. However, on exam her lung exam was unremarkable and she had no peripheral edema. Her CXR was also unremarkable. Given her ARF, her diuretics were held. On transfer to BMT, she was restarted on her outpatient dose of Lasix since she had some trace LE edema and b/l pulmonary crackles. Her Ace-inhibitor was still held because of her resolving ARF. By time of discharge, her renal impairment had resolved to baseline and she will resume her home dose of ACEi. * Hypotension: Likely due to PEs. She was initially treated empirically with Vancomycin and Zosyn for concern of sepsis given the high lactate. Her lactate decreased to 1.1 on repeat ABG. She was never febrile and her vitals stabilized so the antibiotics were d/c'd on day 2 of hospitalization. Blood and urine cultures are still pending, but show no growth. Her BP meds (Diltiazem and Lisinopril) were held, but a Metoprolol 12.5 was started on [**10-5**] because her blood pressure had improved and because of her cardiac risk factors. . 3 Leukocytosis: Pt's WBC trended down. There was no bandemia and no signs of infection. Etiology is unclear, but patient did recently complete a recurring treatment of Decadron. . 4 Acute Renal Failure: Admission Cr was 3.4 and her baseline Cr is 1.1. All diuretics were held as was Metformin for concern of lactic acidosis in the setting of ARF. Her creatinine trended downward for the remainder of her hospitalization. By time of discharge, her Cr resolved to normal. 5 Hyperglycemia: Secondary to decadron. Outpatient Metformin was held because of ARF. FS were done QID with humalog/SSI until all blood sugars were normal. She did not require insulin while on the medical floor. . 6 Multiple Myeloma: Hem/Onc fellow was notified that the patient had been admitted and the decision was made to hold Thalidomide for now. Pt. was transferred to BMT service on [**10-5**]. She will follow-up with her primary hematologist after discharge. . 7 Anemia: Baseline Hct is in the 30s, on iron as an outpatient. Pt. received 1 [**Location **] on [**10-4**] for Hct of 28.5 which appropriately responded to 30.1. . 8 Prophylaxis: PPI (on steroids), heparin gtt, bowel regimen. . 9 FEN: low Na diabetic renal diet. check albumin . 10 Access: PIVs x 2 . 11 Code Status: Full, discussed with pt and family Medications on Admission: Triamterene/HCTZ 37.5/25 mg po daily Aspirin 81 mg PO daily Lasix 40 mg PO Lisinopril 20 mg PO daily Diltiazem XR 480 mg PO Metformin 850 mg PO BID Prilosec 40 QD Thalidomide 100 mg PO QHS Dexamethasone pulse Q2 weeks Epogen 40,000 units zometa Qmonth T#3 1-2 tabs prn TID Diprosone 0.05% cream Discharge Medications: 1. Prilosec 40 mg Capsule, Delayed Release(E.C.) Sig: One (1) Capsule, Delayed Release(E.C.) PO once a day. 2. Outpatient Lab Work Please get the following blood tests checked 2 days after being discharged: PT, PTT, INR. 3. Lisinopril 20 mg Tablet Sig: One (1) Tablet PO once a day. 4. Warfarin 5 mg Tablet Sig: asdir Tablet PO at bedtime: alternated taking 1 tablet and 1 1/2 tablets at night. start 1 tablet on [**2112-10-11**]. Disp:*60 Tablet(s)* Refills:*2* Discharge Disposition: Home Discharge Diagnosis: Primary: Pulmonary Embolism Deep Venous Thrombosis Acute Renal Failure . Secondary: Multiple Myeloma Discharge Condition: good. normal oxygenation on room air. renal function resolved to baseline. tolerating oral nutrition and medication. ambulating unassisted. Discharge Instructions: If you develop any concerning symptoms particularly shortness of breath, chest pain, worsening leg swelling, or signs of severe bleeding, seek medical care. . You need to have your blood checked twice a week to make sure the coumadin dose is the right one for you. You have an appointment in Dr.[**Name (NI) 6168**] clinic on Thursday to get your blood drawn. Weigh yourself every morning, [**Name8 (MD) 138**] MD if weight > 3 lbs. Adhere to 2 gm sodium diet Fluid Restriction to less than 2 liters per day. Followup Instructions: 1) Cardiology Clinic with Dr. [**First Name8 (NamePattern2) 487**] [**Last Name (NamePattern1) **] on [**2112-11-15**] at 3:40pm. Please call [**Telephone/Fax (1) 920**] to confirm. 2) [**Hospital **] Clinic with Dr. [**First Name8 (NamePattern2) **] [**Last Name (NamePattern1) 1557**] on [**2112-10-13**] at 12pm. 3) Please call Dr.[**Name (NI) 25189**] office at [**Telephone/Fax (1) 3581**] to schedule an appointment within the next 2 weeks.	"

In [50]:
# clinical_note = "NURSING PROGRESS NOTES 0800-1900 57 YR OLD MALE W/ HX OF GASTRIC STAPLING [**2119**], THYROIDECTOMY [**2147**] FOR BENIGN GROWTH, CPAP AT HOME RECENT GASTRIC BYPASS [**3-26**] DC'D FROM [**Hospital **] HOSP. [**3-29**]. STARTED W/ LEFT CALF PAIN WHEN WALKING ON SATURDAY, SOB ON SUNDAY AND SYNCOPED ON MONDAY TO [**Hospital **] HOSP THEN TO [**Hospital1 95**] FOR FURTHER TX. CT CHEST: POSITIVE FOR PULMANARY ARTERY EMBOLI, LEFT. ADMITTED TO MICU A RM 4 AT 0830. PT ALREADY ON HEPARIN GTT [**2149**] UNITS/HR. AT 1200 TO ULTRA SOUND FOR VASCULAR US OF BILAT LOWER EXTREM. PT POSITIVE FOR POPLITEAL CLOT OF LEFT LEG. REVIEW OF SYSTEMS: NEURO: PT A &O X3 MAE PERL, T MAX 98.8 RESP: PT WAS ON 6L NC O2 WHEN FIRST ARRIVED FROM [**Location **] SATS 98%, PRESENTLY ON 3L NC SATS 97%, RR 25-30, LS CLEAR UPPER LOBES W/ DIMINISHED BASES. PT DOES IS 5 X Q 1 HR PER SURGERY ( WHO WILL FOLLOW PT D/T RECENT GASTRIC BYPASS). PT HAS PRODUCTIVE COUGH WITH MINIMAL SECRETIONS NOTED. WIFE REPORTS HIS SPUTUM WAS BLD TINGED, HOWEVER SPUTUM NOTED BY THIS RN WAS CLEAR AND THIN. PT HAS RIGHT SIDED LATERAL CHEST PAIN THAT RADIATES TO HIS BACK AND IS WORSE W/ INSPIRATION, #5 ON PAIN SCALE. AFTER TYLENOL 650MG LIQUID, PAIN DOWN TO #2. HOWEVER, AFTER IS STARTED PAIN UP TO A #4 MS [**Last Name (Titles) 7528**] 10MG GIVEN W/ GOOD RELIEF. CV: PT [**Name (NI) 1805**] [**Name (NI) **] 90-100S, THERE IS SOME REPORT THAT HE WAS IN AFIB FOR A SHORT TIME IN THE AMBULANCE WHEN BEING TX TO THIS ED BUT NO HISTORY OF AFIB. HRT SOUNDS S1S2, W /OCCASIONAL PVC. [**2154-4-2**] 5:46 PM IVC GRAM/FILTER Clip # [**Clip Number (Radiology) 112448**] Reason: SADDLE PE/LARGE LT LEG DVT Admitting Diagnosis: PULMONARY EMBOLUS Contrast: OPTIRAY Amt: 40 ********************************* CPT Codes ******************************** * [**Numeric Identifier 751**] INTERUP IVC [**Numeric Identifier 12080**] INTRO CATH SVC/IVC * * -51 MULTI-PROCEDURE SAME DAY [**Numeric Identifier 752**] PERC PLCMT IVC FILTER * * C1769 GUID WIRES INCL INF C1769 GUID WIRES INCL INF * * C1880 VENA CAVA FILTER [**Hospital 2**] MEDICAL CONDITION: 57M with saddle PE, large L LE DVT. REASON FOR THIS EXAMINATION: Needs IVC filter FINAL REPORT INDICATION: A 57-year-old with left superficial femoral vein DVT and large saddle pulmonary embolus. IVC filter is requested for prophylaxis of further pulmonary emboli. PHYSICIANS: The procedure was performed by Dr. [**First Name (STitle) 48843**] [**Name (STitle) 48844**] and Dr. [**First Name4 (NamePattern1) 34428**] [**Last Name (NamePattern1) **]. Dr. [**First Name (STitle) **], the attending radiologist, was present and supervising during the entire procedure. PROCEDURE/FINDINGS: Signed written informed consent was obtained from the patient prior to the procedure. In addition, a pre-procedure time-out was performed prior to the procedure confirming patient identity, the procedure to be performed, and the side and site of entry. Approximately 5 cc of 1% Lidocaine was injected into the right groin for local anesthesia. The right common femoral vein was visualized using both ultrasonographic and fluoroscopic guidance. The common femoral vein was then accessed using a 19-gauge micro-puncture needle through which a Glide-wire was then passed into the inferior vena cava. The micro-puncture needle was removed and a 4-French Omni-Flush catheter was then advanced over the guide wire to the level of the distal inferior vena cava just proximal to the iliac bifurcation. An inferior vena cavogram was then performed. The inferior vena cavogram demonstrated a patent inferior vena cava and patent bilateral common iliac veins. Reflux of contrast was seen into the right and left iliac veins consistent with elevated right atrial pressures. A suitable position for filter placement was then selected with the tip of the filter at the margin of the inferior endplate of the L1 vertebral body, below the inflow of the renal veins. The glide wire was removed and a 0.035 [**Last Name (un) 52**] guide wire was advanced through the catheter. The catheter was then exchanged for a 7-French Bard recovery filter system. The guide wire was removed. The recovery filter was then deployed successfully into the designated area using fluoroscopic guidance. (Over) [**2154-4-2**] 5:46 PM IVC GRAM/FILTER Clip # [**Clip Number (Radiology) 112448**] Reason: SADDLE PE/LARGE LT LEG DVT Admitting Diagnosis: PULMONARY EMBOLUS Contrast: OPTIRAY Amt: 40 ______________________________________________________________________________ FINAL REPORT (Cont) Final x-ray imaging were obtained and demonstrated the recovery filter to be in good position and proper orientation. The sheath was then removed and pressure was held on the right groin until hemostasis was achieved. COMPLICATIONS: None. ANESTHESIA: 1% Lidocaine for local anesthesia, 2 mg of IV Versed, and 50 mcg of IV Fentanyl. IMPRESSION: 1. IVC venogram demonstrating a single and patent IVC and patent bilateral common iliac veins. Reflux of contrast is seen into the iliac veins consistent with elevated right atrial pressures. 2. Successful placement of a Bard recovery IVC filter within the inferior vena cava inferior to the level of the renal veins. The filter could be removed at any time using a jugular approach if necessary. CONT. NURSING PROGRESS NOTES 0830-1900 CV CONT: HEPARIN GTT PRESENTLY INFUSING AT 1550 UNITS/HR, REPEAT PTT PENDING. PEDAL PULSES +3 POST TIBS +2. PT PRESENTLY IN IR FOR IVC FILTER PLACEMENT D/T FINDINGS OF THE VASCULAR US: LG CLOT NOTED IN LEFT LEG EXTENDING FROM HIP TO KNEE. PT [**Name (NI) 57**] 1 LITER OF NS W/ 20MEQ KCL @100CC/HR. PT EKG POS FOR S1/Q3/T3 W/ T WAVE INVERSIONS IN THE PRECORDIAL LEADS. ECHO DONE. GI: PT HAS NOT BEEN EATING OR DRINKING FOR THE LAST 2 DAYS HE STATES \" HE DID NOT FEEL LIKE HE WAS HUNGRY\". BS + ABD SOFT AND OBESE. PT IS ON HIS STAGE 3 DIET (ALL INFO RE DIET CAN BE FOUND IN FRONT OF THE BLUE MED BOOK- COPIED FROM WIFE'S INFORMATION) SURGERY WILL BE FOLLOWING PT RE: S/P GASTRIC BYPASS. PT IS NPO W/ ICE CHIPS. GU: FOLEY CATH DRAINING YELLOW URINE SKIN: MIDLINE CHEST SCAR NOTED W/ STERI STIPS INTACT. PT IS IN A BIG BOY BED. SOCIAL: PT HAS WIFE VEDA AND DAUGHTER WHO HAS RECENTLY MOVED TO [**State **] CODE: FULL ID: PT IS [**Name (NI) 7529**] DIABETIC PER PT, AND IS ON SSC REGULAR INSULIN PLAN: 1.PLACE IVC FILTER 2. CONT HEPARIN GTT GOAL 60-100 AND CHECK F/U PTT FROM 1730 3. CONT TO ENCOURAGE IS Q1 HR WA 4. ? C/O IN AM 5. IF ABLE TO TAKE PO- USE GUIDELINES AS NOTED IN BLUE BOOK 6. NEED FURTHER ORDER FOR MAINTANCE FLUID 7. PT MUST HAVE LIQUID MEDS AND HE TAKES LEVOXYL AS A PILL AND LETS IT MELT UNDER HIS TONGUE W/ A SIP OF WATER. 8. PLEASE KEEP PT AND WIFE UP TO DATE W/ PLAN OF CARE Sinus tachycardia Right bundle branch block No previous tracing for comparison PATIENT/TEST INFORMATION: Indication: Pulmonary embolus. Right ventricular function. Shortness of breath. Height: (in) 71 Weight (lb): 351 BSA (m2): 2.68 m2 BP (mm Hg): 114/62 HR (bpm): 102 Status: Inpatient Date/Time: [**2154-4-2**] at 11:04 Test: Portable TTE (Complete) Doppler: Full doppler and color doppler Contrast: Definity Technical Quality: Adequate INTERPRETATION: Findings: LEFT VENTRICLE: Moderate symmetric LVH. Normal LV cavity size. Hyperdynamic LVEF. No resting LVOT gradient. No LV mass/thrombus. RIGHT VENTRICLE: Dilated RV cavity. RV function depressed. Abnormal systolic septal motion/position consistent with RV pressure overload. No RV mass/thrombus. AORTA: Normal aortic root diameter. AORTIC VALVE: Normal aortic valve leaflets (3). No AS. No AR. No AS. MITRAL VALVE: Normal mitral valve leaflets with trivial MR. TRICUSPID VALVE: Normal tricuspid valve leaflets. Mild [1+] TR. PERICARDIUM: Trivial/physiologic pericardial effusion. Conclusions: There is moderate symmetric left ventricular hypertrophy. The left ventricular cavity size is normal. Left ventricular systolic function is hyperdynamic (EF>75%). No masses or thrombi are seen in the left ventricle. The right ventricular cavity is dilated. Right ventricular systolic function appears depressed. There is abnormal systolic septal motion/position consistent with right ventricular pressure overload. There is no mass/thrombus in the right ventricle. The aortic valve leaflets (3) appear structurally normal with good leaflet excursion and no aortic regurgitation. There is no aortic valve stenosis. The mitral valve appears structurally normal with trivial mitral regurgitation. There is at least moderate pulmonary hypertension.There is a trivial/physiologic pericardial effusion. IMPRESSION: These findings are consistent with significant RV strain. Sinus tachycardia Right bundle branch block Since previous tracing of the same date, sinus tachycardia rate decreased"

print(check_PE(clinical_note_2639_157999))
print(check_PE(clinical_note_18718_162712))
print(check_PE(clinical_note_17977_198062))
print(check_PE(clinical_note_1066_154406))

# clinical_note = "[**2123-10-19**] 1:11 PM CTA CHEST W&W/O C &RECONS; CT 100CC NON IONIC CONTRAST Clip # [**Clip Number (Radiology) 74441**] Reason: SOB ? PE Field of view: 36 Contrast: OPTIRAY Amt: 100 ______________________________________________________________________________ FINAL ADDENDUM Outside radiographs of the chest obtained from [**Hospital3 30270**] Hospital obtained on [**2123-11-20**] again reveal extensive pleural calcification unchanged to that shown on the CT scan. [**2123-10-19**] 1:11 PM CTA CHEST W&W/O C &RECONS; CT 100CC NON IONIC CONTRAST Clip # [**Clip Number (Radiology) 74441**] Reason: SOB ? PE Field of view: 36 Contrast: OPTIRAY Amt: 100 ______________________________________________________________________________ [**Hospital 4**] MEDICAL CONDITION: 83 year old man s/p ERCP w/ sphincterotomty yesterday, now w/ acute hypotension, sinus tach and RBB - need to rule in PE to initiate anticoagulation. REASON FOR THIS EXAMINATION: ? acute PE No contraindications for IV contrast ______________________________________________________________________________ FINAL REPORT INDICATION: S/P ERCP. Patient has acute hypotension, tachycardia and new RBBB. TECHNIQUE: Helical contiguous axial images were obtained from the lung bases to the level of the thoracic inlet. 100cc of Optiray was injected in fast bolus (CTA). High resolution ct of the lungs. CTA CHEST W&W/0 CONTRAST: The right side of the heart is enlarged. A large filling defect is observed in the right main pulmonary artery. There are also filling defects in the right upper lobe artery, in the artery to the middle lobe in the interlobar and segmental arteries of the right lobe. There are filling defects in the main pulmonary artery to the left lower lobe and its branches. The findings are consistent with extensive PE. There is a pericardial effusion. No pleural effusion is seen. The mediastinum is normal. The lung windows reveal small passive atelectasis in the left upper lobe and in the lingula. Multiple linear opacities in right and left upper lobes. Also seen, is bilateral pleural thickening with multiple linear calcifications inside it, more apparent in the upper part of the left side. There is pneumobilia consistent with s/p ERCP. There is a large hiatal hernia IMPRESSION: 1) Extensive bilateral PE. 2) Secondary changes in the lungs which are consistent with multiple atelectasis and fibrotic changes. 3) Pleural thickening with calcifications which is observed bilaterally. This finding may be consistent with previous infection, TB infection and previous exposure to asbestosis. 4) Big hiatal hernia. (Over) [**2123-10-19**] 1:11 PM CTA CHEST W&W/O C &RECONS; CT 100CC NON IONIC CONTRAST Clip # [**Clip Number (Radiology) 74441**] Reason: SOB ? PE Field of view: 36 Contrast: OPTIRAY Amt: 100 ______________________________________________________________________________ FINAL REPORT (Cont) NURSING ADMISSION NOTE 1200-1500 PT IS AN 83 YO MALE WITH HO ALZHEIMERS DEMETIA CONTROLLED WITH ARICEPT WHO TX FROM [**Hospital3 **] ON [**10-13**] FOR W/U OF ?PANCRREATITIS. PT HAD GENERALIZED WEAKNESS, N/V AND RUQ PAIN. PMH ALSO INCLUDES CHRONIC UTI'S ?COPD, HYPERCHOL, AND GALLSTONES. PT WAS TO BE D/CED HOME BUT C/O ABD PAIN AND U/S SHOWED STONES . ERCP DONE YESTERDAY [**10-18**] WITH SPHINCTEROTOMY AND REMOVAL OF CBD STONES. DID WELL POST OP UNTIL THIS AM WHEN PT BECAME DIAPHORETIC , DECREASE O2 SAT, DECREASE BP. SENT TO ICU FOR MANAGEMENT. UPON ARRIVAL HERE TODAY, PT HAD L SUBCLAVIAN TRIPLE LUMEN PLACED, AND DOWN TO CT SCAN WHICH REVEALED BILATERAL PE'S. STARTING ON HEPARIN GTT WITHOUT BOLUS. ROS--- CARDIAC--BP 110/70. HR ST 110-120 WITHOUT OBSERVED VEA. IVF AT 125 HR. BLOOD CX SENT X2. TRIPLE LUMEN SUBCLAVIAN PLACED. HCT AND CPK'S SENT RESP--WEARING O2 AT 100%NRB WITH SAO2 97-98%. SPONT RESP 18-26.REGULAR. LLL WITH VERY DIMINISHED BREATH SOUNDS AND CLEAR IN OTHER LOBES. NON-PRODUCTIVE COUGH. GI--NPO AT PRESENT. GU--DECREASE UO 15 CC OVER 2 HRS. HO AWARE. URINE IS AMBER. SKIN--L GLUTEAL CHEEK HAS RASH WHICH PT'S WIFE STATES THAT HE WEARS DEPENDS AT NIGHT. NO OTHER BREAKDOWN NOTED. ENDO--UNREMARKABLE AT PRESENT. COPING--WIFE AND KIDS IN TO SEE PT. THEY ARE AWARE OF DX OF PE. ID--BLOOD CX X2. HYPOTHERMIC AT PRESENT. DISPO--FULL CODE. A--PE'S BILATERAL P--CON'T TO OFFER SUPPORT TO FAMILY . ASSESS RESP STATUS. CHECK PT/PTT AT 9PM Probable sinus tachycardia Low QRS voltages in precordial leads Right bundle branch block Nonspecific ST-T wave changes P-R interval 0.12 Since previous tracing of [**2123-10-15**]: above changes present	"
# print(check_PE(clinical_note))

False
True
True
True


In [ ]:
!unzip "/content/patients_datewise_gpt35-20231210T201923Z-001.zip"

In [51]:
import os
import glob
import pandas as pd


folder_name = 'patients_datewise_gpt35_negation_handled_pe/'
os.makedirs(folder_name, exist_ok=True)


gpt_output_dir = 'patients_datewise_gpt35'
gpt_csv_files = glob.glob(os.path.join(gpt_output_dir, '*.csv'))

ctr = 0
cnt = 0

for csv_file in gpt_csv_files:
    df = pd.read_csv(csv_file)
    date = ""

    try:
      for idx, visit in df.iterrows():
        # Check if pulmonary embol is present and not a negation
        if check_PE(visit['TEXT']):
          date = visit['CHARTDATE']
          break

      # date = df[df['TEXT'].str.contains('pulmonary embolism', case=False)]['CHARTDATE'].iloc[0]
      if date != "":
        df['FIRST_OCCURRENCE_CORRECTED'] = df.apply(compare_dates, args=(date, ), axis=1)
        df.to_csv(folder_name + 'fo_' + str(csv_file.split('/')[-1]), index=False)
        cnt += 1
      else:
        df['FIRST_OCCURRENCE_CORRECTED'] = df['FIRST_OCCURANCE']
    except:
      df['FIRST_OCCURRENCE_CORRECTED'] = df['FIRST_OCCURANCE']
      ctr += 1

print(cnt)
print(ctr)

428
0


In [52]:
!zip -r "/content/patients_datewise_gpt35_negation_handled_pe.zip" "/content/patients_datewise_gpt35_negation_handled_pe"

  adding: content/patients_datewise_gpt35_negation_handled_pe/ (stored 0%)
  adding: content/patients_datewise_gpt35_negation_handled_pe/fo_gpt35_16k_fo_text_20411_130603.csv (deflated 71%)
  adding: content/patients_datewise_gpt35_negation_handled_pe/fo_gpt35_16k_fo_text_31792_144955.csv (deflated 74%)
  adding: content/patients_datewise_gpt35_negation_handled_pe/fo_gpt35_16k_fo_text_23792_143170.csv (deflated 72%)
  adding: content/patients_datewise_gpt35_negation_handled_pe/fo_gpt35_16k_fo_text_10017_199207.csv (deflated 69%)
  adding: content/patients_datewise_gpt35_negation_handled_pe/fo_gpt35_16k_fo_text_15105_138421.csv (deflated 67%)
  adding: content/patients_datewise_gpt35_negation_handled_pe/fo_gpt35_16k_fo_text_26125_107965.csv (deflated 68%)
  adding: content/patients_datewise_gpt35_negation_handled_pe/fo_gpt35_16k_fo_text_28032_124627.csv (deflated 71%)
  adding: content/patients_datewise_gpt35_negation_handled_pe/fo_gpt35_16k_fo_text_28097_140501.csv (deflated 73%)
  add